## Performance on describing final layer neurons of ResNet-50 (ImageNet)

In [ ]:
import os
#virtually move to parent directory
os.chdir("..")

import torch
import numpy as np 
print(np.__version__)
import pandas as pd
from sentence_transformers import SentenceTransformer

import clip
import utils
import similarity

## Arguments for CLIP-Dissect

In [ ]:
clip_name = 'ViT-B/16'
#target_name = 'resnet-18-places-1'
target_name = 'deit-tiny-relu'
target_layer = 'head'#blocks[11].mlp.act'
batch_size = 100
device = 'cuda'
pool_mode = 'avg'

similarity_fn = similarity.soft_wpmi

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')
clip_model, _ = clip.load(clip_name, device=device)

with open('data/imagenet_labels.txt', 'r') as f: 
    imagenet_classnames = (f.read()).split('\n')

## Run CLIP-Dissect

In [ ]:
rows = [("imagenet_val", "broden_labels_clean.txt"),
       ("imagenet_val", "3k.txt"),
       ("imagenet_val", "10k.txt"),
       ("imagenet_val", "20k.txt"),
       ("imagenet_val", "imagenet_labels.txt"),
       ("cifar100_train", "20k.txt"),
       ("broden", "20k.txt"),
       ("imagenet_broden", "20k.txt"),]

In [ ]:
for d_probe, concept_set in rows:
    folder_name = '{}_FINAL/{}'.format(target_name, d_probe)
    save_dir = 'experiments/{}/saved_activations'.format(folder_name)
   

    with open('data/concept_sets/' + concept_set, 'r') as f: 
        words = (f.read()).split('\n')
    utils.save_activations(clip_name = clip_name, target_name = target_name, target_layers = [target_layer], 
                           d_probe = d_probe, concept_set = concept_set, batch_size = batch_size, 
                           device = device, pool_mode=pool_mode, save_dir = save_dir)

    save_names = utils.get_save_names(clip_name = clip_name, target_name = target_name,
                                      target_layer = target_layer, d_probe = d_probe,
                                      concept_set = concept_set, pool_mode=pool_mode,
                                      save_dir = save_dir)

    target_save_name, clip_save_name, text_save_name = save_names

    similarities, target_feats = utils.get_similarity_from_activations(target_save_name, clip_save_name, 
                                                        text_save_name, similarity_fn, device=device)

    clip_preds = torch.argmax(similarities, dim=1)
    clip_preds = [words[int(pred)] for pred in clip_preds]

    clip_cos, mpnet_cos = utils.get_cos_similarity(clip_preds, imagenet_classnames, clip_model, model, device, batch_size)
    print("D_probe:{}, Concept set:{}".format(d_probe, concept_set))
    print("CLIP-Dissect - Clip similarity: {:.4f}, mpnet similarity: {:.4f}".format(clip_cos, mpnet_cos))

## Baselines

In [ ]:
netdissect_res = pd.read_csv('data/NetDissect_results/resnet50_imagenet_fc.csv')
nd_preds = netdissect_res['label'].values

clip_cos, mpnet_cos = utils.get_cos_similarity(nd_preds, imagenet_classnames, clip_model, model, device, batch_size)
print("Network Dissection - Clip similarity: {:.4f}, mpnet similarity: {:.4f}".format(clip_cos, mpnet_cos))

In [ ]:
milan_preds = pd.read_csv('data/MILAN_results/m_base_resnet50_imagenet.csv')
milan_preds = milan_preds[milan_preds['layer']=='fc']
milan_preds = milan_preds.sort_values(by=['unit'])
milan_preds = list(milan_preds['description'])

clip_cos, mpnet_cos = utils.get_cos_similarity(milan_preds, imagenet_classnames, clip_model, model, device, batch_size)
print("MILAN - Clip similarity: {:.4f}, mpnet similarity: {:.4f}".format(clip_cos, mpnet_cos))